### Land Surface Temperature, Urban Heat Island (UHI), Urban Thermal Field variance Index (UTFVI)

Dataset Used: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2

### IMPORTS

In [14]:
import time
from typing import Any

import ee
import geemap
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

### AUTHENTICATE AND INITIALIZE

In [15]:
# Authenticate to the Earth Engine servers
ee.Authenticate()

# Initialize the Earth Engine module.
ee.Initialize()

### VARIABLES

In [16]:
DATASET_NAME: str = "LANDSAT/LC08/C02/T1_L2"
START_DATE: str = "2023-01-01"
END_DATE: str = "2024-01-01"
BANDS: list[str] = ["SR_B4", "SR_B3", "SR_B2"]
SCALE: int = 30
# ZONES: range = range(2, 10)
TARGET_ZONE = 9
ZONES = [4, 9]
MAXPIXELS = 1e10
OPACITY_LEVEL = 0.4 
MIN_TEMP = -40  # Minimum temperature in Celsius
MAX_TEMP = 60   # Maximum temperature in Celsius

### FUNCTIONS

In [17]:
def mask_landsat_clouds(image):
  """Masks clouds in a landsat image using the QA_PIXEL band.
  Args:
      image (ee.Image): A landsat image.
  Returns:
      ee.Image: A cloud-masked landsat image.
  """
  qa = image.select('QA_PIXEL')

  # Bits 3 and 4 are clouds and Cloud Shadow, respectively.
  cloud_bit_mask = 1 << 3
  cloud_shadow_bit_mask = 1 << 4

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))
  )
  return image.updateMask(mask).select("S.*").copyProperties(image, ["system:time_start"])

  # return image.updateMask(mask).divide(10000)

In [18]:
# Applies scaling factors.
def apply_scale_factors(image):
    optical_bands = image.select("SR_B.").multiply(0.0000275).add(-0.2)
    thermal_bands = image.select("ST_B.*").multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

### GET COLLECTION

In [19]:
shapefile_path = f"/vsizip/../shape_files/zone_{TARGET_ZONE}.zip/layers/POLYGON.shp"
gdf = gpd.read_file(shapefile_path)
roi = geemap.gdf_to_ee(gdf)

In [20]:
# roi.geometry()

In [21]:
LANDSAT = ee.ImageCollection(DATASET_NAME)

LANDSAT_ROI = LANDSAT.filter(ee.Filter.date(START_DATE, END_DATE)) \
    .filter(ee.Filter.bounds(roi.geometry())) \
    .map(apply_scale_factors) \
    .map(mask_landsat_clouds) \
    .median()

NDVI = LANDSAT_ROI.normalizedDifference(["SR_B5", "SR_B4"]).rename("ndvi") # NIR and Red Bands

### CALCULATE NDVI STATISTICS

In [22]:
NDVI_MIN = ee.Number(NDVI.reduceRegion(
    reducer = ee.Reducer.min(),
    geometry = roi.geometry(),
    scale = SCALE,
    maxPixels = MAXPIXELS
).values().get(0))

NDVI_MAX = ee.Number(NDVI.reduceRegion(
    reducer = ee.Reducer.max(),
    geometry = roi.geometry(),
    scale = SCALE,
    maxPixels = MAXPIXELS
).values().get(0))

# Fraction of vegetation
FV = NDVI.subtract(NDVI_MIN).divide(NDVI_MAX.subtract(NDVI_MIN)).pow(ee.Number(2)).rename("FV")

# Emissivity
EM = FV.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename("EM")

# Select the thermal band
THERMAL = LANDSAT_ROI.select("ST_B10").rename("THERMAL")

# Calculate Land Surface Temperature (LST)
LST = THERMAL.expression(
    "(tb / (1 + (0.00115 * (tb / 0.48359547432)) * log(em))) - 273.15",
    {
        "tb": THERMAL.select("THERMAL"),
        "em": EM
    }
).rename("LST")

# image = image.addBands(ndvi)

### Urban Heat Island

In [23]:
LST_MEAN = ee.Number(LST.reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = roi.geometry(),
    scale = SCALE,
    maxPixels = MAXPIXELS
).values().get(0))

LST_STD = ee.Number(LST.reduceRegion(
    reducer = ee.Reducer.stdDev(),
    geometry = roi.geometry(),
    scale = SCALE,
    maxPixels = MAXPIXELS
).values().get(0))

# Print mean and standard deviation of LST in AOI
print("#" * 50)
print('Mean LST in ROI:', LST_MEAN.getInfo())
print('STD LST in ROI:', LST_STD.getInfo())
print("#" * 50)

# Calculate Normalized UHI
UHI = LST.subtract(LST_MEAN).divide(LST_STD).rename('UHI')

##################################################
Mean LST in ROI: 32.87045882491294
STD LST in ROI: 2.62051495377301
##################################################


### Urban Thermal Field Variance Index (UTFVI)

In [24]:
UTFVI = LST.subtract(LST_MEAN).divide(LST).rename('UTFVI')

### VISUALIZATION

In [25]:
from IPython.display import display, HTML
# Visualization parameters for True Colors
vis = {"bands": BANDS, "min": 0.0, "max": 0.3}

# Visualization parameters for NDVI
ndvi_vis = {"min": -1, "max": 1, "palette": ["blue", "white", "green"]}

# Visualization parameters for LST
palette_lst = ["040274", "040281", "0502a3", "0502b8", "0502ce", "0502e6", "0602ff", "235cb1", "307ef3", "269db1", "30c8e2", "32d3ef", "3be285", "3ff38f", "86e26f", "3ae237", "b5e22e", "d6e21f", "fff705", "ffd611", "ffb613", "ff8b13", "ff6e08", "ff500d", "ff0000", "de0101", "c21301", "a71001", "911003"
]
lst_vis = {"min": -40, "max": 60, "palette": palette_lst}

# Visualization parameters for UHI
palette_uhi = ["313695", "74add1", "fed976", "feb24c", "fd8d3c", "fc4e2a", "e31a1c", "b10026"]
uhi_vis = {"min": -4, "max": 4, "palette": palette_uhi}

# Visualization parameters for UTFVI
palette_utfvi = ["313695", "74add1", "fed976", "feb24c", "fd8d3c", "fc4e2a", "e31a1c", "b10026"]
utfvi_vis = {"min": -1, "max": 0.3, "palette": palette_utfvi}

# Set opacity level between 0 (transparent) and 1 (opaque)
opacity_level = OPACITY_LEVEL  
# Draw boundaries of the shapefile region loaded in 'table'
boundary = ee.FeatureCollection(roi)

Map = geemap.Map()
# Map = geemap.Map(center=[45.49995129887764, 9.18765328746192], zoom=13)
# Map.set_center(9.18765328746192, 45.49995129887764, 13)
Map.centerObject(roi.geometry(), 13)
Map.add_layer(LANDSAT_ROI, vis, "True Color (432)", False)
Map.add_layer(NDVI, ndvi_vis, "NDVI Layer", False)
Map.addLayer(LST, lst_vis, "LST Layer", False)
Map.addLayer(UHI, uhi_vis, "UHI Layer", True)
Map.addLayer(UTFVI, utfvi_vis, "UTFVI Layer", False)
Map.addLayer(boundary, {"color": "000000"}, "Region Boundary", True, opacity_level)
Map.addLayerControl()

# Legend setup
min_temp = MIN_TEMP  # Minimum temperature in Celsius
max_temp = MAX_TEMP   # Maximum temperature in Celsius
num_colors = len(palette_lst)
temp_interval = (max_temp - min_temp) / num_colors
temp_ranges = [f"{min_temp + i * temp_interval:.1f} to {min_temp + (i + 1) * temp_interval:.1f} °C" for i in range(num_colors)]

legend_dict = dict(zip(temp_ranges, ["#" + color for color in palette_lst]))

# Display a title above the map
display(HTML(f"<h3>Urban Heat Island: Milan-Zone {TARGET_ZONE}</h3>"))
Map.add_legend(title=f"TempRange (°C) Milan-Zone {TARGET_ZONE}", legend_dict=legend_dict)

Map

Map(center=[45.512724879429385, 9.180956342696671], controls=(WidgetControl(options=['position', 'transparent_…

### Download Median Image

In [26]:
for zone in tqdm(ZONES):
    print(f"Processing and exporting data for Zone {zone}")
    shapefile_path = f"/vsizip/../shape_files/zone_{zone}.zip/layers/POLYGON.shp"
    gdf = gpd.read_file(shapefile_path)
    roi = geemap.gdf_to_ee(gdf)

    # Ensure all steps that depend on the ROI are within the loop
    LANDSAT_ROI = LANDSAT.filter(ee.Filter.date(START_DATE, END_DATE)) \
        .filter(ee.Filter.bounds(roi.geometry())) \
        .map(apply_scale_factors) \
        .map(mask_landsat_clouds) \
        .median()

    # Recalculate NDVI, FV, EM, THERMAL, LST, UHI, and UTFVI
    # Example only shows NDVI; add similar steps for other indices and values
    NDVI = LANDSAT_ROI.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI")  # Update band names if necessary
    
    # NDVI STATISTICS
    NDVI_MIN = ee.Number(NDVI.reduceRegion(
    reducer = ee.Reducer.min(),
    geometry = roi.geometry(),
    scale = SCALE,
    maxPixels = MAXPIXELS
    ).values().get(0))

    NDVI_MAX = ee.Number(NDVI.reduceRegion(
        reducer = ee.Reducer.max(),
        geometry = roi.geometry(),
        scale = SCALE,
        maxPixels = MAXPIXELS
    ).values().get(0))

    # Fraction of vegetation
    FV = NDVI.subtract(NDVI_MIN).divide(NDVI_MAX.subtract(NDVI_MIN)).pow(ee.Number(2)).rename("FV")
    
    # Emissivity
    EM = FV.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename("EM")
    
    # Select the thermal band
    THERMAL = LANDSAT_ROI.select("ST_B10").rename("THERMAL")
    
    # Calculate Land Surface Temperature (LST)
    LST = THERMAL.expression(
        "(tb / (1 + (0.00115 * (tb / 0.48359547432)) * log(em))) - 273.15",
        {
            "tb": THERMAL.select("THERMAL"),
            "em": EM
        }
    ).rename("LST")

    # URBAN HEAT ISLAND
    LST_MEAN = ee.Number(LST.reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = roi.geometry(),
    scale = SCALE,
    maxPixels = MAXPIXELS
    ).values().get(0))

    LST_STD = ee.Number(LST.reduceRegion(
        reducer = ee.Reducer.stdDev(),
        geometry = roi.geometry(),
        scale = SCALE,
        maxPixels = MAXPIXELS
    ).values().get(0))

    # Calculate Normalized UHI
    UHI = LST.subtract(LST_MEAN).divide(LST_STD).rename('UHI')

    # Urban Thermal Field Variance Index (UTFVI)
    UTFVI = LST.subtract(LST_MEAN).divide(LST).rename('UTFVI')

    # Convert all derived bands to the same type, here Float64 for example
    NDVI = NDVI.toDouble()
    FV = FV.toDouble()
    EM = EM.toDouble()
    THERMAL = THERMAL.toDouble()
    LST = LST.toDouble()
    UHI = UHI.toDouble()
    UTFVI = UTFVI.toDouble()

    # Combine all bands into one image for export
    combined_image = LANDSAT_ROI.addBands([NDVI, FV, EM, THERMAL, LST, UHI, UTFVI])

    # EXPORTING THE COMBINED IMAGE
    print(f"EXPORTING MEDIAN IMAGES FOR Zone {zone}")
    task = ee.batch.Export.image.toDrive(
        image=combined_image.select(["NDVI", "FV", "EM", "THERMAL", "LST", "UHI", "UTFVI"]),
        description=f"Combined_Image_Zone{zone}",
        fileNamePrefix=f"zone{zone}_combined_{START_DATE}_{END_DATE}",
        # folder="NDVI_LST_UHI_UTFVI_roi_median_30m",
        folder="LST_UHI_UTFVI_roi_median_30m",
        scale=SCALE,
        region=roi.geometry().getInfo()["coordinates"],
        maxPixels=MAXPIXELS
    )
    task.start()
    while task.active():
        print("Polling for task (id: {}).".format(task.id))
        time.sleep(10)  # Poll every 10 seconds

    status = task.status()
    print("#" * 50)
    print(status)
    if 'error_message' in status:
        print("Failed with error:", status['error_message'])
    else:
        print(f"Task for Zone {zone}, combined image download completed with state {task.status()['state']}: ")
    print("#" * 50)

  0%|          | 0/2 [00:00<?, ?it/s]

Processing and exporting data for Zone 4
EXPORTING MEDIAN IMAGES FOR Zone 4
Polling for task (id: OWU5NSMYKUY3XXPCSONV7EC3).
Polling for task (id: OWU5NSMYKUY3XXPCSONV7EC3).
Polling for task (id: OWU5NSMYKUY3XXPCSONV7EC3).
##################################################
{'state': 'COMPLETED', 'description': 'Combined_Image_Zone4', 'priority': 100, 'creation_timestamp_ms': 1717431191375, 'update_timestamp_ms': 1717431217444, 'start_timestamp_ms': 1717431198186, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1jBVVvYzyv_1pngwmF31lt8VV9ZGiUn4t'], 'attempt': 1, 'batch_eecu_usage_seconds': 17.472177505493164, 'id': 'OWU5NSMYKUY3XXPCSONV7EC3', 'name': 'projects/earthengine-legacy/operations/OWU5NSMYKUY3XXPCSONV7EC3'}


 50%|█████     | 1/2 [00:33<00:33, 33.80s/it]

Task for Zone 4, combined image download completed with state COMPLETED: 
##################################################
Processing and exporting data for Zone 9
EXPORTING MEDIAN IMAGES FOR Zone 9
Polling for task (id: T3N6JV36XNAAECBK5QWN5KL7).
Polling for task (id: T3N6JV36XNAAECBK5QWN5KL7).
Polling for task (id: T3N6JV36XNAAECBK5QWN5KL7).
##################################################
{'state': 'COMPLETED', 'description': 'Combined_Image_Zone9', 'priority': 100, 'creation_timestamp_ms': 1717431225302, 'update_timestamp_ms': 1717431248595, 'start_timestamp_ms': 1717431233865, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1jBVVvYzyv_1pngwmF31lt8VV9ZGiUn4t'], 'attempt': 1, 'batch_eecu_usage_seconds': 12.270427703857422, 'id': 'T3N6JV36XNAAECBK5QWN5KL7', 'name': 'projects/earthengine-legacy/operations/T3N6JV36XNAAECBK5QWN5KL7'}


100%|██████████| 2/2 [01:09<00:00, 34.87s/it]

Task for Zone 9, combined image download completed with state COMPLETED: 
##################################################


### DOWNLOAD ALL IMAGES (skip where NDVI is not valid)

In [14]:
from tqdm import tqdm

skipped_images = 0
failed_images = []

for zone in tqdm(ZONES):
    print(f"Processing and exporting data for Zone {zone}")
    shapefile_path = f"/vsizip/../shape_files/zone_{zone}.zip/layers/POLYGON.shp"
    gdf = gpd.read_file(shapefile_path)
    roi = geemap.gdf_to_ee(gdf)

    LANDSAT_ROI = LANDSAT.filter(ee.Filter.date(START_DATE, END_DATE)) \
        .filter(ee.Filter.bounds(roi.geometry())) \
        .map(apply_scale_factors) \
        .map(mask_landsat_clouds)

    
    def process_image(image):
        NDVI = image.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI").toDouble()
        
        NDVI_MIN = ee.Number(NDVI.reduceRegion(reducer=ee.Reducer.min(), geometry=roi.geometry(), scale=SCALE, maxPixels=MAXPIXELS).get('NDVI', 0))
        NDVI_MAX = ee.Number(NDVI.reduceRegion(reducer=ee.Reducer.max(), geometry=roi.geometry(), scale=SCALE, maxPixels=MAXPIXELS).get('NDVI', 0))
    
        FV = NDVI.subtract(NDVI_MIN).divide(NDVI_MAX.subtract(NDVI_MIN)).pow(ee.Number(2)).rename("FV").toDouble()
        EM = FV.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename("EM").toDouble()
        THERMAL = image.select("ST_B10").rename("THERMAL").toDouble()
        LST = THERMAL.expression("(tb / (1 + (0.00115 * (tb / 0.48359547432)) * log(em))) - 273.15", \
                {"tb": THERMAL.select("THERMAL"), "em": EM}).rename("LST").toDouble()
        
        return image.addBands([NDVI, FV, EM, THERMAL, LST])
        

    processed_collection = LANDSAT_ROI.map(process_image)

    print(f"EXPORTING ALL IMAGES FOR Zone {zone}")
    image_ids = processed_collection.aggregate_array("system:index").getInfo()
    print("Total Images: ", len(image_ids))

    for i, image_id in enumerate(tqdm(image_ids, desc="Exporting All Images")):
        try:
            image = ee.Image(processed_collection.filter(ee.Filter.eq("system:index", image_id)).first())
            task = ee.batch.Export.image.toDrive(
                image=image.select(["NDVI", "FV", "EM", "THERMAL", "LST"]),
                description="Image Export {}".format(i + 1),
                fileNamePrefix=f"zone{zone}_NDVI_ST_B10_{image.id().getInfo()}",
                folder="NDVI_LST_UHI_UTFVI_ALL_IMAGES_30m",
                scale=SCALE,
                region=roi.geometry().getInfo()["coordinates"],
                maxPixels=MAXPIXELS,
            )
            task.start()

            while task.active():
                print("Polling for image (id: {}).".format(task.id))
                time.sleep(10)  # Poll every 10 seconds

            status = task.status()
            if status['state'] != 'COMPLETED':
                raise Exception("Image failed to export")
            else:
                print(f"Image {i + 1} completed with state: {status['state']}")

        except Exception as e:
            print(f"Image {i + 1} Failed: ", str(e))
            skipped_images += 1
            failed_images.append(image_id)

    print("#" * 50)
    print(f"All images for Zone {zone} have been processed and initiated for download.")
    print(f"Number of skipped images: {skipped_images}")
    print("Failed image IDs:", failed_images)
    print("#" * 50)

  0%|          | 0/2 [00:00<?, ?it/s]

Processing and exporting data for Zone 4
EXPORTING ALL IMAGES FOR Zone 4
Total Images:  84



Exporting All Images:   0%|          | 0/84 [00:00<?, ?it/s]

Polling for image (id: GAZQJAIT2HTZL3M5NAECM2EU).
Polling for image (id: GAZQJAIT2HTZL3M5NAECM2EU).



Exporting All Images:   1%|          | 1/84 [00:23<32:17, 23.34s/it]

...

Image 82 completed with state: COMPLETED
Polling for image (id: IM3RXI2HPJZKK26EU4BJMTKZ).
Polling for image (id: IM3RXI2HPJZKK26EU4BJMTKZ).



Exporting All Images:  99%|█████████▉| 83/84 [25:55<00:23, 23.54s/it]

Image 83 completed with state: COMPLETED
Polling for image (id: LGIHDL7REUA2ZACLM7KG4ZUT).
Polling for image (id: LGIHDL7REUA2ZACLM7KG4ZUT).



100%|██████████| 2/2 [51:44<00:00, 1552.16s/it]

Image 84 completed with state: COMPLETED
##################################################
All images for Zone 9 have been processed and initiated for download.
Number of skipped images: 44
Failed image IDs: ['LC08_193028_20230227', 'LC08_193028_20230502', 'LC08_193028_20230518', 'LC08_193028_20230721', 'LC08_193028_20231228', 'LC08_193028_20240317', 'LC08_193028_20240504', 'LC08_193029_20230211', 'LC08_193029_20230227', 'LC08_193029_20230331', 'LC08_193029_20230502', 'LC08_193029_20230518', 'LC08_193029_20230923', 'LC08_193029_20231212', 'LC08_193029_20231228', 'LC08_193029_20240129', 'LC08_193029_20240317', 'LC08_193029_20240418', 'LC08_193029_20240504', 'LC08_194028_20230407', 'LC08_194028_20230610', 'LC08_194028_20230829', 'LC08_194028_20231016', 'LC08_193028_20230227', 'LC08_193028_20230502', 'LC08_193028_20230518', 'LC08_193028_20230721', 'LC08_193028_20231228', 'LC08_193028_20240317', 'LC08_193028_20240504', 'LC08_193029_20230227', 'LC08_193029_20230331', 'LC08_193029_20230502'

### DOWNLOAD IMAGES (ONLY NDVI, ST_B10 BANDS ONLY)

In [ ]:
# for zone in tqdm(ZONES):
#     print(f"Processing and exporting data for Zone {zone}")
#     shapefile_path = f"/vsizip/../shape_files/zone_{zone}.zip/layers/POLYGON.shp"
#     gdf = gpd.read_file(shapefile_path)
#     roi = geemap.gdf_to_ee(gdf)

#     LANDSAT_ROI = LANDSAT.filter(ee.Filter.date(START_DATE, END_DATE)) \
#         .filter(ee.Filter.bounds(roi.geometry())) \
#         .map(apply_scale_factors) \
#         .map(mask_landsat_clouds)

#     def process_image(image):
#         NDVI = image.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI").toDouble()
#         return image.addBands(NDVI)

#     # Map the function over the collection
#     processed_collection = LANDSAT_ROI.map(process_image)

#     # Now exporting each image
#     print(f"EXPORTING ALL IMAGES FOR Zone {zone}")
#     image_ids = processed_collection.aggregate_array("system:index").getInfo()
#     print("Total Images: ", len(image_ids))
#     # print(f"Collection: {image_ids}")

#     for i, image_id in enumerate(tqdm(image_ids, desc="Exporting All Images")):
#         image = ee.Image(processed_collection.filter(ee.Filter.eq("system:index", image_id)).first())
#         task = ee.batch.Export.image.toDrive(
#         **{
#             "image": image.select(["NDVI", "ST_B10"]),
#             "description": "Image Export {}".format(i + 1),
#             "fileNamePrefix": f"zone{zone}_NDVI_ST_B10_{image.id().getInfo()}",
#             "folder": "NDVI_ST_B10_ALL_IMAGES_30m",
#             "scale": SCALE,
#             "region": roi.geometry().getInfo()["coordinates"],  # Use ROI geometry
#             # "region": image.geometry().bounds().getInfo()["coordinates"],
#             "maxPixels": MAXPIXELS,
#         })
#         task.start()

#         # Wait for the task to complete (simplified monitoring)
#         while task.active():
#             print("Polling for image (id: {}).".format(task.id))
#             time.sleep(10)  # Poll every 10 seconds

#         status = task.status()
#         print("#" * 50)
#         print(status)
#         if "error_message" in status:
#             print(f"Image {i} Failed: ", status["error_message"])
#         else:
#             print(f"Image {i + 1} completed with state: {status['state']}")
#         print("#" * 50)

#     print(f"All images for Zone {zone} have been processed and initiated for download.")